In [ ]:
#| default_exp common._modules

In [ ]:
#| hide
%load_ext autoreload
%autoreload 2

# NN Modules

In [ ]:
#| export
import torch
import torch.nn as nn

In [ ]:
#| hide
from nbdev.showdoc import show_doc

In [ ]:
#| export
ACTIVATIONS = ['ReLU','Softplus','Tanh','SELU','LeakyReLU','PReLU','Sigmoid']

# 0. Concentrator

In [ ]:
#| export
class Concentrator(nn.Module):
    def __init__(self,
                 n_series: int,
                 type: str, 
                 treatment_var_name: str,
                 input_size: int,
                 h: int,
                 freq: int,
                 mask_future: bool = True,
                 ):
        super().__init__()
        
        assert type in ['subcutaneous_injection'], "treatment type not available."

        self.n_series = n_series
        self.type = type
        self.treatment_var_name = treatment_var_name
        self.freq = freq

        # K parameter for each time-series
        self.k_a = nn.Embedding(self.n_series, 1)
        # Initialize k_a
        init_k = torch.zeros((self.n_series, 1))
        self.k_a.weight.data.copy_(init_k)
        
        self.input_size = input_size
        self.h = h
        self.mask_future = mask_future
        
    def treatment_concentration(self, t, k_a, sigma=1):
        t = torch.div(t, 60) #60 minutes --> hours # TODO: make more adaptable to different data freq
            
        conc = torch.exp(torch.negative(torch.pow((torch.log(t + 1e-5) - k_a), 2)) /\
               (2 * sigma**2)) / \
               (t * sigma * torch.sqrt(torch.tensor(2) * torch.pi) + 1e-5) 
               # Add small increment (1e-5) or else k_a --> [nan]

        return conc

    def forward(self,
                treatment_exog: torch.Tensor,
                stat_exog: torch.Tensor,
                idx: torch.Tensor) -> torch.Tensor:

        treatment_var = treatment_exog[:, :, -1] # [B,L,C] -> [B,L]

        # Set treatment in forecasting window to 0
        if self.mask_future:
            treatment_var[:, -self.h:] = 0

        b = treatment_var.shape[0]
        l = treatment_var.shape[1]
        
        idx = idx.long()
        # Constrain k_a with sigmoid
        k_a = torch.sigmoid(self.k_a(idx)) # [B, 1, 1] for static k_a      
        
        # Create [B,L,L] matrix
        lt = torch.tensor(range(l))
        ltr = lt.repeat((l, 1)) - lt.reshape(-1, 1)
        ltr[ltr<0]=0

        ltr_batch = torch.zeros((b, l, l)).to(treatment_exog.device)
        ltr_batch[:] = ltr
        
        # Apply frequency
        ltrf_batch = ltr_batch * self.freq
        
        # Multiple concentration by treatement_var (dose)
        conc = self.treatment_concentration(ltrf_batch, k_a)
        scaled_conc = conc*(treatment_var.reshape(b, l, 1))
        treatment = scaled_conc.nansum(dim=1) #[B, L]

        # Replace treatment variable with concentration
        treatment_exog_out = torch.zeros(treatment_exog.shape).to(treatment_exog.device)
        treatment_exog_out[:, :, :-1] += treatment_exog[:, :, :-1]
        treatment_exog_out[:, :, -1] += treatment

        return treatment_exog_out

## 1. MLP

Multi-Layer Perceptron

In [ ]:
#| export
class MLP(nn.Module):
    """Multi-Layer Perceptron Class

    **Parameters:**<br>
    `in_features`: int, dimension of input.<br>
    `out_features`: int, dimension of output.<br>
    `activation`: str, activation function to use.<br>
    `hidden_size`: int, dimension of hidden layers.<br>
    `num_layers`: int, number of hidden layers.<br>
    `dropout`: float, dropout rate.<br>
    """
    def __init__(self, in_features, out_features, activation, hidden_size, num_layers, dropout):
        super().__init__()
        assert activation in ACTIVATIONS, f'{activation} is not in {ACTIVATIONS}'
        
        self.activation = getattr(nn, activation)()

        # MultiLayer Perceptron
        # Input layer
        layers = [nn.Linear(in_features=in_features, out_features=hidden_size),
                  self.activation,
                  nn.Dropout(dropout)]
        # Hidden layers
        for i in range(num_layers - 2):
            layers += [nn.Linear(in_features=hidden_size, out_features=hidden_size),
                       self.activation,
                       nn.Dropout(dropout)]
        # Output layer
        layers += [nn.Linear(in_features=hidden_size, out_features=out_features)]

        # Store in layers as ModuleList
        self.layers = nn.Sequential(*layers)

    def forward(self, x):
        return self.layers(x)

## 2. Temporal Convolutions

For long time in deep learning, sequence modelling was synonymous with recurrent networks, yet several papers have shown that simple convolutional architectures can outperform canonical recurrent networks like LSTMs by demonstrating longer effective memory.

**References**<br>
-[van den Oord, A., Dieleman, S., Zen, H., Simonyan, K., Vinyals, O., Graves, A., Kalchbrenner, N., Senior, A. W., & Kavukcuoglu, K. (2016). Wavenet: A generative model for raw audio. Computing Research Repository, abs/1609.03499. URL: http://arxiv.org/abs/1609.03499. arXiv:1609.03499.](https://arxiv.org/abs/1609.03499)<br>
-[Shaojie Bai, Zico Kolter, Vladlen Koltun. (2018). An Empirical Evaluation of Generic Convolutional and Recurrent Networks for Sequence Modeling. Computing Research Repository, abs/1803.01271. URL: https://arxiv.org/abs/1803.01271.](https://arxiv.org/abs/1803.01271)<br>

In [ ]:
#| export
class Chomp1d(nn.Module):
    """ Chomp1d

    Receives `x` input of dim [N,C,T], and trims it so that only
    'time available' information is used. 
    Used by one dimensional causal convolutions `CausalConv1d`.

    **Parameters:**<br>
    `horizon`: int, length of outsample values to skip.
    """
    def __init__(self, horizon):
        super(Chomp1d, self).__init__()
        self.horizon = horizon

    def forward(self, x):
        return x[:, :, :-self.horizon].contiguous()


class CausalConv1d(nn.Module):
    """ Causal Convolution 1d

    Receives `x` input of dim [N,C_in,T], and computes a causal convolution
    in the time dimension. Skipping the H steps of the forecast horizon, through
    its dilation.
    Consider a batch of one element, the dilated convolution operation on the
    $t$ time step is defined:

    $\mathrm{Conv1D}(\mathbf{x},\mathbf{w})(t) = (\mathbf{x}_{[*d]} \mathbf{w})(t) = \sum^{K}_{k=1} w_{k} \mathbf{x}_{t-dk}$

    where $d$ is the dilation factor, $K$ is the kernel size, $t-dk$ is the index of
    the considered past observation. The dilation effectively applies a filter with skip
    connections. If $d=1$ one recovers a normal convolution.

    **Parameters:**<br>
    `in_channels`: int, dimension of `x` input's initial channels.<br> 
    `out_channels`: int, dimension of `x` outputs's channels.<br> 
    `activation`: str, identifying activations from PyTorch activations.
        select from 'ReLU','Softplus','Tanh','SELU', 'LeakyReLU','PReLU','Sigmoid'.<br>
    `padding`: int, number of zero padding used to the left.<br>
    `kernel_size`: int, convolution's kernel size.<br>
    `dilation`: int, dilation skip connections.<br>
    
    **Returns:**<br>
    `x`: tensor, torch tensor of dim [N,C_out,T] activation(conv1d(inputs, kernel) + bias). <br>
    """
    def __init__(self, in_channels, out_channels, kernel_size,
                 padding, dilation, activation, stride:int=1):
        super(CausalConv1d, self).__init__()
        assert activation in ACTIVATIONS, f'{activation} is not in {ACTIVATIONS}'
        
        self.conv       = nn.Conv1d(in_channels=in_channels, out_channels=out_channels, 
                                    kernel_size=kernel_size, stride=stride, padding=padding,
                                    dilation=dilation)
        
        self.chomp      = Chomp1d(padding)
        self.activation = getattr(nn, activation)()
        self.causalconv = nn.Sequential(self.conv, self.chomp, self.activation)
    
    def forward(self, x):
        return self.causalconv(x)

In [ ]:
show_doc(CausalConv1d, title_level=3)

In [ ]:
#| export
class TemporalConvolutionEncoder(nn.Module):
    """ Temporal Convolution Encoder

    Receives `x` input of dim [N,T,C_in], permutes it to  [N,C_in,T]
    applies a deep stack of exponentially dilated causal convolutions.
    The exponentially increasing dilations of the convolutions allow for 
    the creation of weighted averages of exponentially large long-term memory.

    **Parameters:**<br>
    `in_channels`: int, dimension of `x` input's initial channels.<br> 
    `out_channels`: int, dimension of `x` outputs's channels.<br>
    `kernel_size`: int, size of the convolving kernel.<br>
    `dilations`: int list, controls the temporal spacing between the kernel points.<br>
    `activation`: str, identifying activations from PyTorch activations.
        select from 'ReLU','Softplus','Tanh','SELU', 'LeakyReLU','PReLU','Sigmoid'.<br>

    **Returns:**<br>
    `x`: tensor, torch tensor of dim [N,T,C_out].<br>
    """
    # TODO: Add dilations parameter and change layers declaration to for loop
    def __init__(self, in_channels, out_channels, 
                 kernel_size, dilations,
                 activation:str='ReLU'):
        super(TemporalConvolutionEncoder, self).__init__()
        layers = []
        for dilation in dilations:
            layers.append(CausalConv1d(in_channels=in_channels, out_channels=out_channels, 
                                        kernel_size=kernel_size, padding=(kernel_size-1)*dilation, 
                                        activation=activation, dilation=dilation))
            in_channels = out_channels
        self.tcn = nn.Sequential(*layers)

    def forward(self, x):
        # [N,T,C_in] -> [N,C_in,T] -> [N,T,C_out]
        x = x.permute(0, 2, 1).contiguous()
        x = self.tcn(x)
        x = x.permute(0, 2, 1).contiguous()
        return x

In [ ]:
show_doc(TemporalConvolutionEncoder, title_level=3)